Create a Hive enabled spark session, with a persisted derby metastore and sql scripting enabled. This requires spark 4.0.0.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SvnLocalSpark")\
    .config("spark.sql.warehouse.dir", "../spark-data")\
    .config("spark.hadoop.javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=../hive_metastore_db;create=true")\
    .config("spark.sql.scripting.enabled", True)\
    .enableHiveSupport()\
    .getOrCreate()

sc = spark.sparkContext

print(f"spark {spark.version} {sc.uiWebUrl}")

spark 4.0.0 http://DESKTOP-4GOMK6M:4040


In [2]:
for db in spark.catalog.listDatabases():
    print(f"database: {db.name}")
    for tbl in spark.catalog.listTables(db.name):
        print(f"table:{tbl.name}")
print(f"current database:{spark.catalog.currentDatabase()}")

database: copytest
table:commercial_property
database: default
database: dwh_sales
table:dim_customer
table:dim_product
table:dim_store
table:fact_sales
database: landing
table:commercial_properties
database: sales_demo
table:calculations
table:customers
table:products
table:sales_details
table:sales_orders
table:stores
current database:default


In [3]:
%load_ext sparksql_magic

In [4]:
%%sparksql
BEGIN
    DECLARE choice DOUBLE DEFAULT 3.9;
    DECLARE result STRING;
    IF choice < 2 THEN
      VALUES ('one fish');
    ELSEIF choice < 3 THEN
      VALUES ('two fish');
    ELSEIF choice < 4 THEN
      VALUES ('red fish');
    ELSEIF choice < 5 OR choice IS NULL THEN
      VALUES ('blue fish');
    ELSE
      VALUES ('no fish');
    END IF;
END;

col1
red fish


In [5]:
stm = """
BEGIN
  DECLARE choice DOUBLE DEFAULT 3.9;
  DECLARE result STRING;
  IF choice < 2 THEN
    VALUES ('one fish');
  ELSEIF choice < 3 THEN
    VALUES ('two fish');
  ELSEIF choice < 4 THEN
    VALUES ('red fish');
  ELSEIF choice < 5 OR choice IS NULL THEN
    VALUES ('blue fish');
  ELSE
    VALUES ('no fish');
  END IF;
END;
"""
rst = spark.sql(stm)
rst.show()

+--------+
|    col1|
+--------+
|red fish|
+--------+

